# IFU Spectral Extraction

## Introduction

This notebook demonstrates spectral extraction from a [NIRSpec IFU](https://jwst-docs.stsci.edu/near-infrared-spectrograph/nirspec-observing-modes/nirspec-ifu-spectroscopy) datacube. It is a streamlined version of the [ifu_optimal](https://spacetelescope.github.io/jdat_notebooks/notebooks/ifu_optimal/ifu_optimal.html) notebook from the [JWST Data Analysis Notebooks](https://spacetelescope.github.io/jdat_notebooks/) collection.

* Quicklook extraction with [Cubeviz](https://jdaviz.readthedocs.io/en/latest/cubeviz/index.html)
* Retrieving spectra, spatial regions, and spectral regions from Cubeviz
* Creating and manipulating specutils Spectrum1D objects
    * Slicing
    * Extracting a spectral sub-region
    * Simple arithmetic
* Aperture extraction in a notebook 
* Optimal extraction in a notebook



## Data
A faint (quasar) point source was simulated using the NIRSpec Instrument Performance Simulator (IPS), then run through the Spec2 stage of the [JWST pipeline](https://jwst-pipeline.readthedocs.io/en/latest/).   

## Imports 

* _numpy_ for array math
* _scipy_ for ndcube gaussian smoothing
* _specutils_ for Spectrum1D data model and cube manipulation
* _jdaviz_ for the Cubeviz data visualization tool
* _jwst_ for JWST data models
* _photutils_ to define circular apertures
* _astropy.io_ for reading and writing FITS cubes and images
* _astropy.wcs, units, coordinates_ for defining and reading WCS
* _astropy.stats_ for sigma_clipping
* _astropy.utils_ for downloading files from URLs
* _astropy.units_ for unit definitions
* _astropy.nddata_ for uncertainty object types
* _matplotlib_ for plotting spectra and images

In [ ]:
#Resize notebook to full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import scipy

import specutils
from specutils import Spectrum1D, SpectralRegion
from specutils.manipulation import extract_region, spectral_slab
from specutils.analysis import moment

from jdaviz import CubeViz

from jwst import transforms

from photutils import CircularAperture, SkyCircularAperture, aperture_photometry
from photutils.detection import DAOStarFinder
from photutils.centroids import centroid_com, centroid_1dg, centroid_2dg, centroid_quadratic

from regions import PixCoord, CirclePixelRegion

from astropy.io import fits
from astropy import wcs
from astropy.stats import sigma_clip
from astropy.stats import sigma_clipped_stats
from astropy.utils.data import download_file
import astropy.units as u
from astropy.nddata import StdDevUncertainty, VarianceUncertainty

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm



## Simulated NIRSpec IFU Cube

A point source (quasar) was simulated using the NIRSpec Instrument Performance Simulator (IPS), then run through the JWST Spec2 pipeline. We will use this for our science dataset.

In [ ]:
#Download the cube data from Box (bright quasar)
#filename = "https://stsci.box.com/shared/static/ff0bj31acot1272x5qq2clbmto3s6v4f.fits"

# NIRSpec IFU science data cube (faint quasar)
BoxPath = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/IFU_optimal_extraction/"
filename = BoxPath + "NRS00001-faintQSO-F100LP-G140H-01_1_491_SE_2020-08-25T12h15m00_s3d.fits"

cube_file = download_file(filename, cache=True)


## Visualize Science Data with Cubeviz

In [ ]:
cubeviz = CubeViz()
cubeviz.app

### UI Instructions:
#### Task 1:  Load the cube and view it
* Load science datacube into Cubeviz using the next code cell below
* Click the Display icon in the leftmost image viewer 
* Change the stretch to Logarithmic, 99 percentile. 
* Manually adjust the bias and contrast sliders until the faint point source at (x,y) = (17,21) becomes visible
* Scrub through the cube using the Slice slider.

#### Task 2:  Extract a spectrum from a region (subset) of the cube
* Select a circular subset region centered on the source, using the region selection tool under the Tools icon. 
* Note that the region is pixelated and doesn't include fractional pixels.
* Change the collapse method function to "Sum" in spectrum viewer: Display icon: Viewer. 
* Hide the first [SCI] layer by clicking the "eye" icon in Display icon: Layer. This will make the noisy, full-cube spectrum disappear.
* Adjust the size and location of the region until you are happy with the extraction that shows up in the spectrum viewer.

#### Task 3: Select a spectral region for further analysis
* In the spectrum viewer, use the vertical and horizontal pan and zoom tools to focus on the broad H-beta line (redshifted to 1.22 microns).  To pan: drag and drop spectrum with mouse or trackpad. To zoom: scroll with mouse or trackpad.
* Select a spectral subset region in spectrum viewer, using the region selection tool under the Tools icon, centered on a spectral feature of interest.
* Open the Line Analysis plugin to measure various line parameters in spectral subset 2.


## Load Cube into Cubeviz

*Developer notes:* 
* *Multiple spectral_cube warnings occur when loading the cube into Cubeviz*

In [ ]:
#Developer Note--spectral_cube warnings:
cubeviz.app.load_data(cube_file)

## Export Region from Cubeviz
Export the region defined by the user in Cubeviz as an astropy CirclePixel Region, which has units of pixels.  Use "try:, except:" to catch the case where there is no Subset 1 selected in Cubeviz. 

In [ ]:
try:
    region1 = cubeviz.app.get_subsets_from_viewer('spectrum-viewer')['Subset 1']
    print(region1)
    region1_exists = True
    center1_xy = [region1.center.x, region1.center.y]  
    r_pix = region1.radius

except Exception:
    print("There are no regions selected in the cube viewer.")
    region1_exists = False
    center1_xy = [17.1, 20.]
    r_pix = 6.0
      

## Retrieve Subset Spectrum from Cubeviz Spectrum Viewer
Get the spectrum of user-defined Subset1 from the Spectrum Viewer as a Spectrum1D object. Trim to remove bad wavelength ranges.

*Developer notes:* 
* *Can't convert Subset1 Spectrum1D wavelengths from meters to microns.*

In [ ]:
#Check what data are available
#print(cubeviz.app.data_collection)
print(cubeviz.app.get_data_from_viewer('spectrum-viewer'))

# try..except in case there is no Subset1
try:
    spec1d_subset1_untrimmed = cubeviz.app.get_data_from_viewer('spectrum-viewer')['Subset 1']
    print(spec1d_subset1_untrimmed)
    
except Exception:
    print("There are no subsets selected in the spectrum viewer.")
    
#Developer Note--Attribute Error trying to convert to microns
#spec1d_subset1_untrimmed = spec1d_subset1_untrimmed.with_spectral_unit(u.um)


## Trim the Spectral Cube
Read in the spectral cube as a specutils Spectrum1D object, slice it, and trim the wavelength axis using the spectral_slab convenience function.

*Developer notes:* 
* *Can't print Spectrum1D read with JWST s3d format. See [specutils issue #868]( https://github.com/astropy/specutils/issues/868).*
* *spectral slab gives warning for Subset1 spectrum: 'No Observer defined on WCS, Spectral Coord'*

In [ ]:
#Load the cube as a specutils Spectrum1D object 
spec1d_untrimmed = Spectrum1D.read(cube_file, format='JWST s3d')
wavelength_untrimmed = spec1d_untrimmed.spectral_axis.value

#Developer Note--can't print the Spectrum1D.  
#See https://github.com/astropy/specutils/issues/868
#print(spec1d_untrimmed)

#Good spectral and spatial regions
wave_trim =[1.0*u.um,1.44*u.um]
x_trim = [2,-1]
y_trim = [5,-4]

#Trim the spectral and spatial axes in one go
#Spectrum1D objects can be sliced like numpy arrays...
#...and/or use the convenience function specutils.manipulation.spectral_slab
spec1d = spectral_slab(spec1d_untrimmed, wave_trim[0], wave_trim[1])[x_trim[0]:x_trim[1],y_trim[0]:y_trim[1],:]
wavelength = spec1d.spectral_axis
spec1d_len = len(wavelength.value)
print('Trimmed spectral axis:')
print(wavelength)

# Trim the Subset1 spectrum to match
#Developer note--"No observer defined on WCS":
spec1d_subset1 = spectral_slab(spec1d_subset1_untrimmed, wave_trim[0], wave_trim[1])
 
#Adjust extraction region location in trimmed cube
center_trim = PixCoord(x=center1_xy[0]-x_trim[0], y=center1_xy[1]-y_trim[0])
center_x = center_trim.x
center_y = center_trim.y
region_trim = CirclePixelRegion(center=center_trim, radius=r_pix)
print()
print('Adjusted Extraction Region:')
print(region_trim)


## Extract Spectrum in Expanding Circular Aperture (Cone)
This method is appropriate for a point source PSF with width proportional to wavelength. In the next cell, we read the cube into a Spectrum1D object and perform aperture photometry on each wavelength slice using photutils.


In [ ]:
#Use photutils.aperture_photometry to measure flux in expanding or constant aperture
aperture = 'cone'  # 'cone' or 'cylinder'
flux_sum = []
for idx in range(spec1d_len):
    if aperture == 'cone':
        r_cone = r_pix * wavelength.value[idx]/ wavelength.value[0]
    else: r_cone = r_pix  
    aperture_cone = CircularAperture((center_x,center_y), r=r_cone)
    phot_table = aperture_photometry(spec1d.flux.value[:, :, idx], aperture_cone)
    flux_sum.append(phot_table['aperture_sum'][0])
    
cone = Spectrum1D(flux=np.array(flux_sum)*u.MJy/u.sr, spectral_axis=spec1d.spectral_axis)
print(cone)

# Compute the RMS-summed error over the full FOV, in each spectral bin
err_sum = np.sqrt(np.sum(spec1d_untrimmed.uncertainty.array**2, axis=(0,1)))
print("Max RMS Uncertainty: ", np.amax(np.nan_to_num(err_sum)))

#Compare conical extraction to Cubeviz pixelated cylindrical extraction
f, (ax1,ax2) = plt.subplots(2, 1, figsize=(10, 8)) 
ax1.set_title("Spectral extractions")
ax1.set_xlabel("Observed Wavelength (microns)")  
ax1.set_ylabel("Flux Density")
ax1.set_xlim(0.99, 1.45)
ax1.set_ylim(0, 0.4)
ax1.plot(wavelength, cone.flux.value, label="Cone", c='darkorange', alpha=0.5)
try:
    ax1.plot(spec1d_subset1.spectral_axis.value*1.0E6, spec1d_subset1.flux.value, c='r', label="Subset1", alpha=0.4)
except Exception:
    print("There is no Cubeviz Subset1 spectrum to plot.")
ax1.legend()

ax2.set_xlabel("Observed Wavelength (microns)")  
ax2.set_ylabel("Uncertainty")
ax2.plot(wavelength_untrimmed, err_sum, c='r', label="RMS Uncertainty", alpha=0.4)
ax2.set_xlim(0.99, 1.45)
ax2.set_ylim(0, 0.1)
ax2.legend()

plt.show()

Top: Comparison of the conical and Cubeviz subset spectral extractions. 
The conical extraction captures slightly more flux at long wavelengths.
Red-shifted Broad Balmer and narrow [O III] lines  are visible in the quasar spectra. 

Bottom: RMS uncertainty, sqrt(sum(sigma^2)), summed over full IFU footprint. 

## WebbPSF Model PSF for Optimal Extraction
Generate PSF model cube using WebbPSF for NIRSpec IFU, or read in precomputed PSF model cube.

Caution! The WebbPSF model takes about 10 hr to run. Uncomment the following cell to do so. Otherwise, read in the precomputed WebbPSF model, which covers the full F100LP/G140H wavelength range (blue and red). For other filter/grating combinations, uncomment and run the cell below using the wavelengths from the science data set.

In [ ]:
'''
#WebbPSF imports
%pylab inline
import webbpsf

#WebbPSF commands used to create PSF model cube
ns.image_mask = "IFU"  # Sets to 3x3 arcsec square mask
ns = webbpsf.NIRSpec()
wavelengths = wavelength*1.0E-6
psfcube = ns.calc_datacube(wavelengths, fov_pixels=30, oversample=4,  add_distortion=True)
psfcube.writeto("Webbpsf_ifucube.fits")
'''

In [ ]:
BoxPath = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/IFU_optimal_extraction/"
psf_filename = BoxPath+"Webbpsf_ifucube.fits"

# Load with astropy.fits.open
with fits.open(psf_filename, memmap=False) as hdulist:
    psf_model = hdulist['DET_SAMP'].data
    psf_hdr = hdulist['DET_SAMP'].header
    hdulist.info()    
print(psf_model.shape)


## Align Model PSF Cube with Science Data

Find the (x,y) offset between the Model and simulated PSF peaks. Shift and smooth the model PSF cube to align with the simulated data. 

In [ ]:
# Flip model PSF left-right because WebbPSF is flipped with respect to the IPS simulation.
psf_model = psf_model[:, ::-1, :]

#Measure centroids of source and PSF model
#Developer note--centroid_1dg and centroid_2dg give errors here; centroid_quadratic is not robust:
cube_sum = moment(spec1d, order=0)
psf_model_sum = np.sum(psf_model, axis=0)
x1, y1 = centroid_com(cube_sum)
x2, y2 = centroid_com(psf_model_sum)
shiftx = x1 - x2
shifty = y1 - y2
print((x1,y1))
print((x2,y2))

#Manual adjustment
#For some reason, the centroiding is not perfect...
shiftx = shiftx - 0.1
shifty = shifty + 0.5
print("shift = ", (shiftx, shifty))

#Shift model PSF using linear interpolation
#The x and y shifts are interchanged, per python indexing convention
psf_model_aligned = scipy.ndimage.shift(psf_model, (0.0, shifty, shiftx), order=1, 
                                        mode='constant', cval=0.0, prefilter=True)

#Smooth model PSF
#EMSM smoothing for G140H grating
scalerad = 0.046  # sigma (arcsec)
pixelscale = 0.1
scalerad_pix = scalerad / pixelscale
smooth_psf = False
if smooth_psf:
    psf_model_smoothed = scipy.ndimage.filters.gaussian_filter(psf_model_aligned, 
                                                           (0.0, scalerad_pix, scalerad_pix), 
                                                           order=0, mode='reflect', cval=0.0,  
                                                           truncate=10.0)
else:
    psf_model_smoothed = psf_model_aligned
    
    
#PSF Spectrum1D
#We transpose the datacube because the last axis of a Spectrum1D flux array must be the spectral axis
#Trim the PSF model in the wavelength dimension to match the data 
spec1d_webb = Spectrum1D(flux=np.transpose(psf_model_smoothed[0:1872, :, :])*u.Jy, spectral_axis=spec1d.spectral_axis)

#PSF sums over wavelength and spaxels
psf_model_sum = np.sum(psf_model_smoothed, axis=0)
psf_model_fnusum = np.sum(psf_model, axis=(1, 2))

# Scale factor for PSF subtraction
scalefactor = np.amax(cube_sum) / np.amax(psf_model_sum)

# Sum model PSF over spaxels


# Plot the collapsed model and data PSFs
f, ([ax1, ax2, ax3], [ax4, ax5, ax6]) = plt.subplots(2, 3, figsize=(10, 10)) 
ms, mew = 15., 2.
ax1.set_title("PSF slice sum")
ax1.imshow(psf_model_sum,origin = 'lower',norm=LogNorm())
ax1.plot(x1, y1, color='red', marker='+', ms=ms, mew=mew)

ax2.set_title("Science Data slice sum")
ax2.imshow(cube_sum, origin='lower', norm=LogNorm()) 
ax2.plot(x1, y1, color='red', marker='+', ms=ms, mew=mew)

ax3.set_title("Data / PSF Ratio")
ax3.imshow(cube_sum / psf_model_sum, origin='lower', norm=LogNorm())

ax4.set_title("PSF Model integrated flux")
ax4.plot(psf_model_fnusum)

ax5.set_title("Data - PSF")
ax5.imshow(cube_sum - scalefactor * psf_model_sum, origin='lower')

#Developer Note--Unit Conversion Error:
#im6 = ax6.imshow(np.log10(np.absolute(cube_sum - scalefactor * psf_model_sum)))
im6 = ax6.imshow(np.absolute(cube_sum - scalefactor * psf_model_sum), norm=LogNorm())
#plt.colorbar(im6)
ax6.set_title("abs(Data - PSF)")

plt.show()

Top row: Collapsed WebbPSF Model PSF, simulated quasar PSF, and their ratio. The ratio will be used for the optimal extraction below. The red cross symbols show the computed centroids of the two aligned PSFs.

Bottom left: The integrated flux decreases from 98% to 97% of the total model flux over the wavelength range of the dataset.

Bottom center, bottom right : Difference and absolute value of difference between the aligned science data and model PSFs.

## Optimal Extraction using WebbPSF Model

Optimal extraction ([Horne 1986, PASP, 98, 609](https://ui.adsabs.harvard.edu/abs/1986PASP...98..609H/abstract)) weights the flux contributions to a spectrum by their signal-to-noise ratio (SNR). Dividing the simulated data by the model PSF gives an estimate of the total flux density spectrum in each spaxel. A weighted average of these estimates over all spaxels yields the optimally extracted spectrum over the cube. In the faint source limit, where the noise is background-dominated, optimal extraction inside a 3-sigma radius can increase the effective exposure time by a factor of 1.69 (Horne et al. 1986). In the bright source limit, where the noise is dominated by the Poisson statistics of the source, optimal extraction is formally identical to a straight sum over spaxels for a perfect PSF model.
An in-flight reference star observation aligned with the science target PSF could be used in place of the WebbPSF model.


In [ ]:
# Mask out bad data using 3-sigma clipping in each slice
# Developers note: Need a function to do sigma-clipping of Spectrum1D
spec1d_norm = spec1d/spec1d_webb
nan_cleaned = np.nan_to_num(spec1d_norm.flux.value)
spec1d_norm_clipped = sigma_clip(nan_cleaned, sigma=3.0, maxiters=5, axis=(0, 1))
badvoxel = np.where(spec1d_norm_clipped == 0)
spec1d_clean = spec1d
spec1d_clean.flux.value[badvoxel] = 0.0

# Variance
spec1d_var = Spectrum1D(flux=spec1d.uncertainty.array*u.Jy, spectral_axis=spec1d.spectral_axis)

#Horne algorithm
spec1d_optimal_weight = spec1d_webb * spec1d_webb / spec1d_var   # SNR^2
weight_sum = np.sum(spec1d_optimal_weight.flux.value, axis=(0,1))
spec1d_optimal_cube = spec1d_webb * spec1d_clean / spec1d_var
spec_optimal = np.sum(spec1d_optimal_cube.flux.value, axis=(0,1)) / weight_sum
spec1d_optimal = Spectrum1D(flux=spec_optimal*u.Jy, spectral_axis=spec1d.spectral_axis)

# Plots
f, (ax1) = plt.subplots(1, 1, figsize=(12, 6)) 
ax1.set_title("Optimal Extraction Comparison")
ax1.set_xlabel("Observed Wavelength (microns)") 
ax1.set_ylabel("Flux Density")
ax1.set_ylim(0, 0.5)
ax1.plot(wavelength, cone.flux, label="Conical Extraction", alpha=0.5)
ax1.plot(wavelength, 1.35 * spec1d_optimal.flux.value, label="1.35 * Optimal")
ax1.legend()

plt.show()


The optimally extracted spectrum generally shows lower noise compared to the conical extraction, for relatively wide conical extraction apertures (r > 2 pixels). The PSF of the IPS-simulated dataset is not perfectly matched to the WebbPSF simulation, giving ~35% loss of flux in the optimally extracted spectrum. Using a reference PSF from IPS gives a higher fidelity optimal extraction. Ideally, one would use the in-flight PSF of a bright reference star for optimal extraction of in-flight data.

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>

Notebook created by Patrick Ogle.